In [1]:
from statistics import mode
import tensorflow
import cv2,dlib
from imutils import face_utils
from keras.models import load_model
import numpy as np
import utils
from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.preprocessor import preprocess_input
import skimage 
from scipy.spatial import ConvexHull
#만약 프롬포트에서 python으로 import tensorflow가 되고 주피터 커널로 실행이 안된다면 pip install ipykernel해서 커널을 설치하고 다시 연결
#pip install tensorflow가 안된다면 지우고 다시해봄 그리고 파이썬을 지웠다가 아나콘다 파이썬 버전과 맞춰줌 pip install h5py와 같이
#안됐던 이유는 아마 텐서플로의 버전과 파이썬 버전때문이 아닌듯 의심, python해서 import tensorflow가 된다면 커널문제이니 재설치등
#h5py\h5.pyx in init h5py.h5() 
#**AttributeError: type object 'h5py.h5.H5PYConfig' has no attribute '__reduce_cython__'**
#오류의 경우 지우고 다시 깔거나 pip install cython 그리고 파이썬에서 바로 확인

잘되네


In [2]:
import os 
print(os.getcwd())

C:\Users\nicky\Desktop\졸프


In [3]:
import sys
print(sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


In [4]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [5]:
# parameters for loading data and images
detector = dlib.get_frontal_face_detector()#기본으로 제공되는 얼굴인식모델
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')#얼굴 랜드마크(68개지점 탐지)
emotion_model_path = 'trained_models/emotion_models/mini_xception.h5'
emotion_labels = get_labels('fer2013')
net = cv2.dnn.readNetFromTorch('models/instance_norm/starry_night.t7')#화풍변경
# hyper-parameters for bounding boxes shape
frame_window = 10
emotion_offsets = (20, 40)

# loading models
# face_detection = load_detection_model(detection_model_path)
emotion_classifier = load_model(emotion_model_path, compile=False)

# getting input model shapes for inference
emotion_target_size = emotion_classifier.input_shape[1:3]

# starting lists for calculating modes
emotion_window = []



In [ ]:
# # video streaming
# cv2.namedWindow('window_frame')
# video_capture = cv2.VideoCapture(0)

# while True:
#     bgr_image = video_capture.read()[1]
#     gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
#     rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
#     faces = detect_faces(face_detection, gray_image)

#     for face_coordinates in faces:

#         x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
#         gray_face = gray_image[y1:y2, x1:x2]
#         try:
#             gray_face = cv2.resize(gray_face, (emotion_target_size))
#         except:
#             continue

#         gray_face = preprocess_input(gray_face, True)
#         gray_face = np.expand_dims(gray_face, 0)
#         gray_face = np.expand_dims(gray_face, -1)
#         emotion_prediction = emotion_classifier.predict(gray_face)
#         emotion_probability = np.max(emotion_prediction)
#         emotion_label_arg = np.argmax(emotion_prediction)
#         emotion_text = emotion_labels[emotion_label_arg]
#         emotion_window.append(emotion_text)

#         if len(emotion_window) > frame_window:
#             emotion_window.pop(0)
#         try:
#             emotion_mode = mode(emotion_window)
#         except:
#             continue
            
#         if emotion_text == '0':
#             color = emotion_probability * np.asarray((255, 0, 0))
#             emotion_mode='angry'
#         elif emotion_text == '1':
#             color = emotion_probability * np.asarray((0, 0, 255))
#             emotion_mode='disgust'
#         elif emotion_text == '2':
#             color = emotion_probability * np.asarray((255, 255, 0))
#             emotion_mode='fear'
#         elif emotion_text == '3':
#             color = emotion_probability * np.asarray((0, 255, 255))
#             emotion_mode='happy'
#         elif emotion_text == '4':
#             color = emotion_probability * np.asarray((0, 255, 255))
#             emotion_mode='neutral'
#         elif emotion_text == '5':
#             color = emotion_probability * np.asarray((0, 255, 255))
#             emotion_mode='sad'
#         else:
#             color = emotion_probability * np.asarray((0, 255, 0))
#             emotion_mode='surprise'

#         color = color.astype(int)
#         color = color.tolist()

#         draw_bounding_box(face_coordinates, rgb_image, color)
#         draw_text(face_coordinates, rgb_image, emotion_mode,
#                   color, 0, -45,2, 5)

#     bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)
    
    
#     MEAN_VALUE = [103.939, 116.779, 123.680]
#     blob = cv2.dnn.blobFromImage(bgr_image, mean=MEAN_VALUE)

#     net.setInput(blob)
#     output = net.forward()

#     output = output.squeeze().transpose((1, 2, 0))

#     output += MEAN_VALUE
#     output = np.clip(output, 0, 255)
#     output = output.astype('uint8')
    
#     cv2.imshow('window_frame', output)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# video_capture.release()
# cv2.destroyAllWindows()

In [6]:
from imutils import face_utils
import imutils
import numpy as np
import collections
import dlib
import cv2
def face_remap(shape):
    remapped_image = cv2.convexHull(shape)
    return remapped_image
image = cv2.imread('images/07.jpg')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

out_face = np.zeros_like(image)

# initialize dlib's face detector (HOG-based) and then create the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# detect faces in the grayscale image
rects = detector(gray, 1)

# loop over the face detections
for (i, rect) in enumerate(rects):
    
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)
    

   #initialize mask array
    remapped_shape = np.zeros_like(shape) 
    feature_mask = np.zeros((image.shape[0], image.shape[1]))   

   # we extract the face
    remapped_shape = face_remap(shape)
    cv2.fillConvexPoly(feature_mask, remapped_shape[0:27], 1)# 얼굴영역 다각형으로 그리기
    feature_mask = feature_mask.astype(np.bool)#탐지한 얼굴 영역
    print('a',feature_mask.shape)
    out_face[feature_mask] = image[feature_mask]#영역에 대한 부분을 원래 부분으로 가져다 씌움
    print(out_face.shape)
    cv2.imshow("mask_inv", out_face)
    
cv2.waitKey(0)
cv2.destroyAllWindows()
    

C:\Users\nicky\AppData\Local\Temp\ipykernel_12696\314317141.py:37: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  feature_mask = feature_mask.astype(np.bool)#탐지한 얼굴 영역


a (720, 960)
(720, 960, 3)
a (720, 960)
(720, 960, 3)
a (720, 960)
(720, 960, 3)
a (720, 960)
(720, 960, 3)
a (720, 960)
(720, 960, 3)
a (720, 960)
(720, 960, 3)
a (720, 960)
(720, 960, 3)


In [7]:
def face_remap(shape):
    remapped_image = cv2.convexHull(shape)
    return remapped_image

In [10]:
def crop_face(detector,predictor,img,gray_img):
    gray=gray_img
    image=img
    rects = detector(gray, 1)
    out_face = np.zeros_like(img)
    # loop over the face detections
    for (i, rect) in enumerate(rects):
    
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)


       #initialize mask array
        remapped_shape = np.zeros_like(shape) 
        feature_mask = np.zeros((image.shape[0], image.shape[1]))   

       # we extract the face
        remapped_shape = face_remap(shape)
        cv2.fillConvexPoly(feature_mask, remapped_shape[0:27], 1)# 얼굴영역 다각형으로 그리기
        feature_mask = feature_mask.astype(np.bool)#탐지한 얼굴 영역
        out_face[feature_mask] = image[feature_mask]
    return out_face,feature_mask



In [11]:
img = dlib.load_rgb_image('images/07.jpg')#face_recognition는 RGB로 파일을 받아옴
bgr_image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)#그래서 opencv에서 사용가능하도록 BGR로 고침
gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
faces = detector(img)
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
print(faces)  # (ys, xs, ye, xe) 오른쪽위좌표,왼쪽 밑좌표

# cv2.imshow('before', cv2.resize(bgr_image,(540,720)))#처리 전

for face_coordinates in faces:
   
    
    y1,x1,y2,x2= face_coordinates.top(),face_coordinates.left(),face_coordinates.bottom(),face_coordinates.right() 
    print('각각좌표',y1,x1,y2,x2)
    gray_face = gray_image[y1:y2,x1:x2]
    
    
    try:
        gray_face = cv2.resize(gray_face, (emotion_target_size))#48,48,1
    except:
        continue
    
    gray_face = preprocess_input(gray_face, True)
    gray_face = np.expand_dims(gray_face, 0)
    gray_face = np.expand_dims(gray_face, -1)
    
    emotion_prediction = emotion_classifier.predict(gray_face)
    emotion_probability = np.max(emotion_prediction)
    emotion_label_arg = np.argmax(emotion_prediction)
    emotion_text = emotion_labels[emotion_label_arg]
    print('감정',emotion_text)
    emotion_window.append(emotion_text)

    if len(emotion_window) > frame_window:
        emotion_window.pop(0)
    try:
        emotion_mode = mode(emotion_window)
    except:
        continue

    if emotion_text == 'angry':
        color = emotion_probability * np.asarray((255, 0, 0))
    elif emotion_text == 'disgust':
        color = emotion_probability * np.asarray((0, 0, 255))
    elif emotion_text == 'fear':
        color = emotion_probability * np.asarray((255, 255, 0))
    elif emotion_text == 'happy':
        color = emotion_probability * np.asarray((0, 255, 255))
    elif emotion_text == 'neutral':
        color = emotion_probability * np.asarray((0, 255, 255))
    elif emotion_text == 'sad':
        color = emotion_probability * np.asarray((0, 255, 255))
        
    else:
        color = emotion_probability * np.asarray((0, 255, 0))


    color = color.astype(int)
    color = color.tolist()
    print('색은',color)
#     cv2.rectangle(bgr_image, (x1,y1), (x2, y2), (0, 0, 255), 5)
    cv2.putText(bgr_image,emotion_text,(x1,y1),cv2.FONT_HERSHEY_SIMPLEX,1,color,3)
    
    #face-crop
    cropped_img=crop_face(detector,predictor,bgr_image,gray_image)[0]
    
    #얼굴영역만 GAN화풍처리
    
    h, w, c = bgr_image.shape
    cropped_img = cv2.resize(cropped_img, dsize=(500, int(h / w * 500)))#비율 조정

    MEAN_VALUE = [103.939, 116.779, 123.680]
    blob = cv2.dnn.blobFromImage(cropped_img, mean=MEAN_VALUE)

    net.setInput(blob)
    output = net.forward()

    output = output.squeeze().transpose((1, 2, 0))

    output += MEAN_VALUE
    output = np.clip(output, 0, 255)
    output = output.astype('uint8')
    
    output = cv2.resize(output, (w, h))
    
    bgr_image[cropped_img[1]]=output[cropped_img[1]]
    
 
    



cv2.imshow('After', cv2.resize(bgr_image,(540,720)))



cv2.waitKey(0)
cv2.destroyAllWindows()
    


rectangles[[(304, 198) (390, 285)], [(327, 377) (476, 526)], [(405, 173) (477, 245)], [(480, 107) (604, 231)], [(538, 251) (796, 509)], [(35, 208) (121, 294)], [(758, 90) (973, 305)]]
각각좌표 198 304 285 390
감정 happy
색은 [0, 251, 251]


C:\Users\nicky\AppData\Local\Temp\ipykernel_12696\3634310646.py:20: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  feature_mask = feature_mask.astype(np.bool)#탐지한 얼굴 영역


각각좌표 377 327 526 476
감정 happy
색은 [0, 150, 150]
각각좌표 173 405 245 477
감정 sad
색은 [0, 93, 93]
각각좌표 107 480 231 604
감정 happy
색은 [0, 147, 147]
각각좌표 251 538 509 796
감정 neutral
색은 [0, 146, 146]
각각좌표 208 35 294 121
감정 neutral
색은 [0, 146, 146]
각각좌표 90 758 305 973
감정 happy
색은 [0, 120, 120]


In [ ]:
# # By image
# img = cv2.imread('images/07.jpg')
# # img=cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)#90도 반시계
# # img=cv2.rotate(img,cv2.ROTATE_90_CLOCKWISE)#90도 시계
# bgr_image = img
# gray_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2GRAY)
# rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)
# faces = detect_faces(face_detection, gray_image)#face 좌표
# print(faces)
# for face_coordinates in faces:

#     x1, x2, y1, y2 = apply_offsets(face_coordinates, emotion_offsets)
#     print('x1, x2, y1, y2',x1, x2, y1, y2)
    
#     gray_face = gray_image[y1:y2, x1:x2]
#     try:
#         gray_face = cv2.resize(gray_face, (emotion_target_size))
#     except:
#         continue
    
#     gray_face = preprocess_input(gray_face, True)
#     gray_face = np.expand_dims(gray_face, 0)
#     gray_face = np.expand_dims(gray_face, -1)
#     emotion_prediction = emotion_classifier.predict(gray_face)
#     emotion_probability = np.max(emotion_prediction)
#     print(emotion_probability)
#     emotion_label_arg = np.argmax(emotion_prediction)#value
#     print(emotion_label_arg)
#     emotion_text = str(emotion_label_arg)
#     print(emotion_text)
#     emotion_window.append(emotion_text)

#     if len(emotion_window) > frame_window:
#         emotion_window.pop(0)
#     try:
#         emotion_mode = mode(emotion_window)
#     except:
#         continue

#     if emotion_text == '0':
#         color = emotion_probability * np.asarray((255, 0, 0))
#         emotion_mode='angry'
#     elif emotion_text == '1':
#         color = emotion_probability * np.asarray((0, 0, 255))
#         emotion_mode='disgust'
#     elif emotion_text == '2':
#         color = emotion_probability * np.asarray((255, 255, 0))
#         emotion_mode='fear'
#     elif emotion_text == '3':
#         color = emotion_probability * np.asarray((0, 255, 255))
#         emotion_mode='happy'
#     elif emotion_text == '4':
#         color = emotion_probability * np.asarray((0, 255, 255))
#         emotion_mode='neutral'
#     elif emotion_text == '5':
#         color = emotion_probability * np.asarray((0, 255, 255))
#         emotion_mode='sad'
#     else:
#         color = emotion_probability * np.asarray((0, 255, 0))
#         emotion_mode='surprise'

#     color = color.astype(int)
#     color = color.tolist()

#     draw_bounding_box(face_coordinates, rgb_image, color)
#     draw_text(face_coordinates, rgb_image, emotion_mode,
#               color, 0, -45, 2, 5)

# bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

# MEAN_VALUE = [103.939, 116.779, 123.680]
# blob = cv2.dnn.blobFromImage(bgr_image, mean=MEAN_VALUE)

# net.setInput(blob)
# output = net.forward()

# output = output.squeeze().transpose((1, 2, 0))

# output += MEAN_VALUE
# output = np.clip(output, 0, 255)
# output = output.astype('uint8')

# print(emotion_window)



# cv2.imshow('before', cv2.resize(img,(540,720)))
# cv2.imshow('window_frame', cv2.resize(output,(540,720)))

# cv2.waitKey(0)
# cv2.destroyAllWindows()